In [4]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
import cudf
print('We will use RAPIDS version',cudf.__version__)
cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)


from coVisitation import CV_B2B, CV_toClicks, CV_carts_orders, ClicktoClick, ClicktoCart, ClicktoOrder, CV_B2B_Q

We will use RAPIDS version 22.10.01+2.gca9a422da9


In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'
! mkdir {outputPath}


mkdir: cannot create directory ‘../output/newSplited/’: File exists


In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

## Train + ValA

In [4]:
TOPN_clicks = 20
TOPN_b2b = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_cartsOrders = 20
SET = 1
note_covisit = ''

In [5]:
files = glob.glob('../input/split_2-1_pqt/*_parquets/*')
saveFolder = outputPath + f'/coVisit_set{SET}_top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
! mkdir {saveFolder}

mkdir: cannot create directory ‘../output/newSplited//coVisit_set1_top20_20_20/’: File exists


In [6]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df

In [7]:
data_cache = {}
for f in files: 
    data_cache[f] = read_file_to_cache(f)

In [9]:
clicktoOrder = ClicktoOrder(files, data_cache, saveFolder)
clicktoOrder.processDisks(20)

cv_C2C = ClicktoCart(files, data_cache, saveFolder)
cv_C2C.processDisks(TOPN_click2cart)

clicktoClick = ClicktoClick(files, data_cache, saveFolder)
clicktoClick.processDisks(TOPN_c2c)

cv_clicks = CV_toClicks(files, data_cache, saveFolder)
cv_clicks.processDisks(TOPN_clicks)

cv_carts_orders = CV_carts_orders(files, data_cache, saveFolder)
cv_carts_orders.processDisks(TOPN_cartsOrders)

cv_B2B = CV_B2B(files, data_cache, saveFolder)
cv_B2B.processDisks(TOPN_b2b)

We will process 101 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 100 in groups of 3...
100 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 100 in groups of 3...
100 , 

### DISK PART 3
Processing files 0 thru 24 in groups of 3...
0 , 3 , 

## Train + Val + local testA

In [5]:
TOPN_clicks = 20
TOPN_b2b = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_cartsOrders = 20
TOPN_b2b_q= 20
SET = 2
note_covisit = ''

In [9]:
files = glob.glob('../input/splited/parquets/*_parquets/*')
# saveFolder = outputPath + f'/coVisit_set{SET}_top{TOPN_b2b}_{TOPN_clicks}_{TOPN_carts}_{TOPN_orders}_{note_covisit}/'
saveFolder = outputPath + f'coVisit/set{SET}/top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
try: 
    os.makedirs(saveFolder) 
except OSError as error: 
    print(error)

[Errno 17] File exists: '../output/newSplited/coVisit/set2/top20_20_20/'


In [10]:
data_cache = {}
for f in files: 
    data_cache[f] = pd.read_parquet(f)

In [11]:
covisiter = CV_B2B_Q(files, data_cache, saveFolder)
covisiter.processDisks(20)

We will process 130 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 129 in groups of 3...
125 , 128 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files

In [7]:
clicktoOrder = ClicktoOrder(files, data_cache, saveFolder)
clicktoOrder.processDisks(20)
cv_C2C = ClicktoCart(files, data_cache, saveFolder)
cv_C2C.processDisks(TOPN_click2cart)
clicktoClick = ClicktoClick(files, data_cache, saveFolder)
clicktoClick.processDisks(TOPN_c2c)
cv_clicks = CV_toClicks(files, data_cache, saveFolder)
cv_clicks.processDisks(TOPN_clicks)
cv_carts_orders = CV_carts_orders(files, data_cache, saveFolder)
cv_carts_orders.processDisks(TOPN_cartsOrders)
cv_B2B = CV_B2B(files, data_cache, saveFolder)
cv_B2B.processDisks(TOPN_b2b)

We will process 130 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 129 in groups of 3...
125 , 128 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files

## Train + Val + Local test + Online testA

In [12]:
TOPN_clicks = 20
TOPN_b2b = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_cartsOrders = 20
SET = 3
note_covisit = ''

In [17]:
files = glob.glob('../input/parquets/*_parquets/*')
# saveFolder = outputPath + f'/coVisit_set3_top{TOPN}/'
saveFolder = outputPath + f'/coVisit/set{SET}/top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
try: 
    os.makedirs(saveFolder) 
except OSError as error: 
    print(error)

[Errno 17] File exists: '../output/newSplited//coVisit/set3/top20_20_20/'


In [14]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df

In [15]:
data_cache = {}
for f in files: 
    data_cache[f] = read_file_to_cache(f)

In [16]:
covisiter = CV_B2B_Q(files, data_cache, saveFolder)
covisiter.processDisks(20)

We will process 146 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 145 in groups of 3...
125 , 128 , 131 , 134 , 137 , 140 , 143 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93

In [10]:
clicktoOrder = ClicktoOrder(files, data_cache, saveFolder)
clicktoOrder.processDisks(20)

We will process 146 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 145 in groups of 3...
125 , 128 , 131 , 134 , 137 , 140 , 143 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93

In [11]:
cv_C2C = ClicktoCart(files, data_cache, saveFolder)
cv_C2C.processDisks(TOPN_click2cart)

We will process 146 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 145 in groups of 3...
125 , 128 , 131 , 134 , 137 , 140 , 143 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93

In [ ]:
clicktoClick = ClicktoClick(files, data_cache, saveFolder)
clicktoClick.processDisks(TOPN_c2c)

In [9]:
cv_C2C = ClicktoClick(files, data_cache, saveFolder)
cv_C2C.processDisks(TOPN_c2c)

We will process 146 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 145 in groups of 3...
125 , 128 , 131 , 134 , 137 , 140 , 143 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93

In [8]:
cv_clicks = CV_toClicks(files, data_cache, saveFolder)
cv_clicks.processDisks(TOPN_clicks)
cv_carts_orders = CV_carts_orders(files, data_cache, saveFolder)
cv_carts_orders.processDisks(TOPN_cartsOrders)
cv_B2B = CV_B2B(files, data_cache, saveFolder)
cv_B2B.processDisks(TOPN_b2b)

We will process 146 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 124 in groups of 3...
100 , 103 , 106 , 109 , 112 , 115 , 118 , 121 , 124 , 
Processing files 125 thru 145 in groups of 3...
125 , 128 , 131 , 134 , 137 , 140 , 143 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93